# A minimial transaction-driven model for personal finance.

## Double-entry accounting

The governing equation behind [double entry accounting](https://en.wikipedia.org/wiki/Double-entry_bookkeeping_system) is:

    Equity = Assets - Liabilities
    
Assuming that all transactions have zero resolution time, it's really that simple — the above holds across all a person's accounts at all times. 

Tracking income and expenses in the above framework is a bit conterintuitive at first: _income-producing_ accounts (e.g. "Salary") work like _equity_ accounts, and _expense-producing_ accounts work like _asset_ accounts. 

As an example, Imagine that I have one job, one credit card, one checking account, and all I spend my money on is food and rent. So my accounts are.
- Checking Account [Asset]
- Credit Card [Liability]
- Salary [Income]
- Rent [Expense]
- Food [Expense]

(I have elected not to count food in my pantry as an asset — if I did, there would be Asset accounts for, "Carrots", "Beans", etc., and an Expense account for "Food Consumption", and perhaps "Spoilage".)

On Monday, I have nothing in my checking account and owe nothing on my credit card. 
- Checking Account = 0
- Credit Card = 0
- Salary = 0
- Rent = 0
- Food =0

On Tuesday, I buy some potatoes and use my credit card to pay for them. My food expense is $5, and my balance on my credit card goes up by $5.
- Checking Account = 0
- Credit Card = 5
- Salary = 0
- Rent = 0
- Food = 5

On Wednesday, my employer direct-deposits $100 into my checking account.
- Checking Account = 100
- Credit Card = 5
- Salary = 100
- Rent = 0
- Food = 5

On Thursday, I have to pay $50 in rent. In this scenario, my landlord deposits my check on the day I write it.
- Checking Account = 50
- Credit Card = 5
- Salary = 100
- Rent = 50
- Food = 5

Finally, on Friday, I pay off my credit card by transferring money in from my checking account.
- Checking Account = 45
- Credit Card = 0
- Salary = 100
- Rent = 50
- Food = 5



In [1]:
import ofxparse
from ofxparse import OfxParser
from uuid import UUID
from datetime import datetime, date

In [2]:
#with open('Chase4395_Activity_20160809.QFX', 'rb') as f:
with open('csp.ofx', 'rb') as f:
    ofx = OfxParser.parse(f)

In [3]:
t0 = ofx.account.statement.transactions[0]
t0

<Transaction units=-74.10>

In [4]:
import gnucashxml
import gzip
from xml.etree import ElementTree

In [5]:
book = gnucashxml.from_filename('fin.gnucash')

In [6]:
len(book.transactions)
UUID(book.transactions[0].splits[0].guid)

UUID('8473d981-67e3-e000-9b29-99d35cb477ce')

In [7]:
accounts = set()
csp_subaccounts = set()
for account, subaccounts, splits in book.walk():
    for split in splits:
        if split.guid in ('a006bf043fa03f345ce8f07d908a631c',
                          '4cc5484f94bf4b02612f2560c1d65eba'):
            t0 = split.transaction
        
#     accounts = accounts.union({account})
#     if account.name == 'Chase Sapphire Preferred':
#         csp_splits = splits

#a = list(accounts)[0]
#print({x.actype for x in accounts})
#print({x.name for x in accounts})
#print({x.name for x in subaccounts})

In [8]:
t0.splits[1].quantity

Decimal('-75.52')

In [9]:
#s = csp_splits[0]

#sorted(csp_splits, key = lambda x: x.transaction.date)[-1].transaction

#s.transaction.guid

In [10]:
def type_check(value, type):
    if type(value) is type:
        return value
    else:
        raise Exception("Expected Type {0}, got type {1}".format(type, type(value)))

class UmatchedAtomicTransaction(object):
    def __init__(self, guid, timestamp, description, account, commodity, quantity):
        self.guid = guid
        self.timestamp = timestamp
        self.description = description
        self.account = account
        self.commodity = commodity
        self.quantity = quantity
    def pair(self, other):
        combined_guid = UUID(int=((self.guid.int + other.guid.int) % 2**128))
        # This is not necessarily the way one would want to come up with an actual time.
        mean_time = self.timestamp + (other.timestamp - self.timestamp)/2
        # We are just discarding one of the descriptions
        description = self.description if self.description != '' else other.description
        if self.quantity > 0 and other.quantity < 0:
            credit = self
            debit = other
        elif self.quantity < 0 and other.quantity > 0:
            debit = self
            credit = other
        else:
            raise Exception("Can't pair Unmatched Transactions with same signs!")
        return AtomicTransaction(
            guid = combined_guid,
            timestamp = mean_time,
            description = description,
            debit_account = debit.account,
            credit_account = credit.account,
            debit_commodity = debit.commodity,
            credit_commodity = credit.commodity,
            debit_registration_date = debit.timestamp,
            credit_registration_date = credit.timestamp,
            debit_quantity = debit.quantity,
            credit_quantity = credit.quantity,
            debit_guid = debit.guid,
            credit_guid = credit.guid
        )
        
class AtomicTransaction(object):
    def __init__(self, guid, timestamp, description, debit_account, credit_account, debit_commodity, credit_commodity, 
                 debit_registration_date, credit_registration_date, debit_quantity, credit_quantity, debit_guid, credit_guid):
        self.guid = guid
        self.timestamp = timestamp
        self.description = description
        self.debit_account = debit_account
        self.credit_account = credit_account
        self.debit_commodity = debit_commodity
        self.credit_commodity = credit_commodity
        self.debit_registration_date = debit_registration_date
        self.credit_registration_date = credit_registration_date
        self.debit_quantity = debit_quantity
        self.credit_quantity = credit_quantity
        self.debit_guid = debit_guid
        self.credit_guid = credit_guid
    # useful for aggregation I think
    def accounts(self):
        return {self.debit_account, self.credit_account}
    @classmethod
    def from_gnucash_transaction(cls, transaction, split_account = ('SPLIT',)):
        
        # This is local to this def because it is specific to accounts of gnucash origin. 
        def get_full_account(account, recurs = tuple()):
            if account.parent is None:
                return (account.name,) + recurs
            else:
                return get_full_account(account.parent,(account.name,)+recurs)
        # This now returns a _list_ of transactions
        uuid = UUID(transaction.guid)
        timestamp = transaction.date_entered
        description = transaction.description
        transaction_date = date(*timestamp.timetuple()[:3])

        debit_splits = [x for x in transaction.splits if x.quantity < 0]
        credit_splits = [x for x in transaction.splits if x.quantity > 0]
        if len(debit_splits) == 0 and len(credit_splits) == 0:
            # This transaction is a no-op
            return list()
        else:
            # If this transaction _isn't_ a no-op, then we need to have at least one debit and at least one credit.
            if len(debit_splits) == 0:
                raise Exception("There seem to be no debit splits.")
            if len(credit_splits) == 0:
                raise Exception("There seem to be no credit splits.")
        # If you have exactly two splits, we are going to emit one transaction. If we have N>2,
        # we are going to emit N transactions.
        if len(debit_splits) == 1 and len(credit_splits) == 1:
            debit_split = debit_splits[0]
            credit_split = credit_splits[0]
            debit_account = get_full_account(debit_split.account)
            credit_account = get_full_account(credit_split.account)
            debit_commodity = debit_split.account.commodity.name
            credit_commodity = credit_split.account.commodity.name
            debit_quantity = debit_split.quantity
            credit_quantity = credit_split.quantity
            debit_guid = UUID(debit_split.guid)
            credit_guid = UUID(credit_split.guid)
            return [cls(
                guid = uuid,
                timestamp = timestamp,
                description = description, 
                debit_account = debit_account,
                debit_commodity = debit_commodity,
                debit_quantity = debit_quantity,
                credit_account = credit_account,
                credit_commodity = credit_commodity,
                credit_quantity = credit_quantity,
                debit_registration_date = transaction_date,
                credit_registration_date = transaction_date,
                debit_guid = debit_guid,
                credit_guid = credit_guid
            )]
        else:
            output = list()
            for s in debit_splits:
                debit_account = get_full_account(s.account)
                debit_commodity = s.account.commodity.name
                debit_quantity = s.quantity
                debit_guid = UUID(s.guid)
                # For splits, the split commodity and quantity always match the non-split side. 
                credit_account = split_account
                credit_commodity = s.account.commodity.name
                credit_quantity = -1*s.quantity
                credit_guid = debit_guid
                output.append(
                    cls(
                        guid = uuid,
                        timestamp = timestamp,
                        description = description,
                        debit_account = debit_account,
                        debit_commodity = debit_commodity,
                        debit_quantity = debit_quantity,
                        credit_account = credit_account,
                        credit_commodity = credit_commodity,
                        credit_quantity = credit_quantity,
                        debit_registration_date = transaction_date,
                        credit_registration_date = transaction_date,
                        debit_guid = debit_guid,
                        credit_guid = credit_guid
                    )
                )
            for s in credit_splits:
                credit_account = get_full_account(s.account)
                credit_commodity = s.account.commodity.name
                credit_quantity = s.quantity
                credit_guid = UUID(s.guid)
                debit_account = split_account
                debit_commodity = s.account.commodity.name
                debit_quantity = -1*s.quantity
                debit_guid = credit_guid
                output.append(
                    cls(
                        guid = uuid,
                        timestamp = timestamp,
                        description = description,
                        debit_account = debit_account,
                        debit_commodity = debit_commodity,
                        debit_quantity = debit_quantity,
                        credit_account = credit_account,
                        credit_commodity = credit_commodity,
                        credit_quantity = credit_quantity,
                        debit_registration_date = transaction_date,
                        credit_registration_date = transaction_date,
                        credit_guid = credit_guid,
                        debit_guid = debit_guid
                    )
                )
            return output




In [11]:
[x.name for x in book.root_account.children]
invest_split_0 = (book.root_account.
 find_account("Investment Accounts (Taxable)").
 find_account("Vanguard Taxable").
 find_account("VTIAX").get_all_splits() 
)[0]
[x.quantity for x in invest_split_0.transaction.splits]
invest = AtomicTransaction.from_gnucash_transaction(invest_split_0.transaction)


In [12]:
# This does work.
book.root_account.find_account("VTIAX")

<Account c698cffadcb489b7bc82f7f1c8d9f2ee>

In [13]:
[current] = [x for x in book.root_account.children if x.name=='Current Assets and Liabilities']

In [14]:
[x.name for x in current.children]

['Liabilities', 'Current Accounts']

In [15]:
liabilities = current.find_account("Liabilities")

In [16]:
[x.name for x in liabilities.children]

['United Chase Visa',
 'Capital One Credit Card',
 'REI Visa, US Bank',
 'Barclaycard',
 'Jetblue Amex',
 'Chase Sapphire Preferred']

In [17]:
csp = liabilities.find_account("Chase Sapphire Preferred")

In [18]:
ss = csp.get_all_splits()[0]

In [19]:
from itertools import chain
transactions = list(chain.from_iterable([AtomicTransaction.from_gnucash_transaction(x) for x in book.transactions]))
len(transactions)

account_hashes = [hash(y) for y in set(chain.from_iterable([x.accounts() for x in transactions]))]
accounts_with_download = ['Current Assets and Liabilities', 'Emergency Fund (Taxable)']
# for t in transactions:
#     for x in t.accounts():
#         if x[1] in accounts_with_download:
#             print(t)
          
transactions_with_download = [t for t in transactions if any( x[1] in accounts_with_download for x in t.accounts() if len(x)>1 )]
len(transactions_with_download)

2672

In [20]:
for x in book.transactions:
    if len(x.splits) > 2:
        long_transaction = x
        break

In [21]:
x.splits[2].guid

'fb513676e68c4d628e4d2f5e37f81f54'

In [22]:
import uuid

uuid.UUID(book.transactions[0].guid)


UUID('1741586b-51ee-c9c1-bc86-cc9a19819658')

In [23]:
[t] = [x for x in book.transactions if x.guid=='1d1690fbd4dce6685aae9f22e4123ea1']


In [24]:
a = AtomicTransaction.from_gnucash_transaction(t)
